# Stats

This notebook loads some data, reports some simple descriptive statistics (means, standard deviationsm etc) and show a number of useful plots (scatter plots, histograms, time series plots).

Most of the descriptive stats use built-in Julia commands. The plots reply on the Plots package and the pdf and quantiles are from the Distributions package (see https://github.com/JuliaStats/Distributions.jl).

## Load Packages

In [1]:
using Distributions      #distributions, random numbers, etc.
                         
include("printmat.jl")   #just a function for prettier matrix printing

println4Ps (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Load Data from Text File

The following is a portion of MyData.csv:

```
date,Mkt-RF,RF,SmallGrowth
197901,4.18,0.77,10.96
197902,-3.41,0.73,-2.09
197903,5.75,0.81,11.71
197904,0.05,0.8,3.27
```

In [3]:
xx   = readdlm("Data/MyData.csv",',',header=true)  #reading the csv file    
x    = xx[1]                                #xx[1] contains the data

println("Column headers: ",xx[2])           #xx[2] contains the headers
println("\nfirst four lines of x:") 
printmat(x[1:4,:])

Column headers: AbstractString["date" "Mkt-RF" "RF" "SmallGrowth"]

first four lines of x:
197901.000     4.180     0.770    10.960
197902.000    -3.410     0.730    -2.090
197903.000     5.750     0.810    11.710
197904.000     0.050     0.800     3.270



### Creating Variables

In [4]:
ym  = x[:,1]                    #yearmonth, like 200712
Rme = x[:,2]                    #picking out the second column
Rf  = x[:,3]                   
R   = x[:,4] +                  # + 0.0 to illustrate that commands continue on the next line
      0.0                     
Re  = R - Rf                    #do R .- Rf if R has several columns (and Rf only one)

println("first 4 obs of Rme and Re")
printmat([Rme[1:4,:] Re[1:4,:]])

y  = round.(Int,ym/100)                 #year
m  = round.(Int,ym - y*100)             #month
dN = Date.(y,m)                         #Date, useful for the plots (below)
printmat(dN[1:4])

first 4 obs of Rme and Re
     4.180    10.190
    -3.410    -2.820
     5.750    10.900
     0.050     2.470

1979-01-01
1979-02-01
1979-03-01
1979-04-01



# Some Descriptive Statistics

## Means and Standard Deviations

The next few cells estimate means, standard deviations, covarances and correlations of the variables Rme (US equity market excess return) and Re (excess returns for a segment of the market, small growth firms). 

In [5]:
μ = mean([Rme Re],1)    #,1 to calculate average along a column, gives a row vector
σ = std([Rme Re],1)     #do \sigma[Tab] to get σ

println("            Rme       Re")
printlnPs("means: ",μ)  #for more stat functions, see the package StatsBase.jl
printlnPs("std:   ",σ)

            Rme       Re
means:      0.602     0.303
std:        4.604     8.572


## Covariances and Correlations

In [6]:
println("\n","cov([Rme Re]): ")          
printmat(cov([Rme Re]))

println("\n","cor([Rme Re]): ")          
printmat(cor([Rme Re]))


cov([Rme Re]): 
    21.197    28.426
    28.426    73.475


cor([Rme Re]): 
     1.000     0.720
     0.720     1.000



## OLS

A simple linear regression
$
y = b x + u
$,
where $x=[1,R^e_m]$.

Clearly, the first element of b is the intercept and the second element is the slope coefficient.

The GLM package (not used here) has powerful regression methods. See https://github.com/JuliaStats/GLM.jl.

In [7]:
c   = ones(size(Rme,1))         #a vector with ones, no. rows from variable
x   = [c Rme]                   #x is a Tx2 matrix
y   = copy(Re)                  #to get standard OLS notation, copy to get an independent copy

b   = inv(x'x)*x'y              #OLS according to a textbook
b2  = x\y                       #also OLS, quicker and numerically more stable
u   = y - x*b                   #OLS residuals
R2a = 1 - var(u)/var(y)         #R2, but that name is already taken

println("OLS coefficients, regressing Re on constant and Rme, different calculations")
printmat([b b2])                
printlnPs("R2: ",R2a) 
printlnPs("no. of observations: ",size(Re,1))

OLS coefficients, regressing Re on constant and Rme, different calculations
    -0.504    -0.504
     1.341     1.341

R2:      0.519
no. of observations:        388


# Drawing Random Numbers and Finding Critical Values

## Random Numbers: Independent Variables

In [8]:
T = 100
x = randn(T,2)    #T x 2 matrix, N(0,1) distribution

println("\n","mean and std of random draws: ")
mu    = mean(x,1)                 
sigma = std(x,1)
printmat([mu;sigma])

println("covariance and correlation matrices:")
printmat(cov(x))
printmat(cor(x))


mean and std of random draws: 
    -0.143    -0.135
     0.998     0.942

covariance and correlation matrices:
     0.996     0.144
     0.144     0.888

     1.000     0.153
     0.153     1.000



## Random Numbers: Correlated Variables

In [9]:
μ = [-1;10]
Σ = [1 0.5;
     0.5 2]

d = MvNormal(μ,Σ)       

T = 100
x = rand(d,T)'          #T x 2 matrix

println("\n","mean and std of random draws: ")
mu    = mean(x,1)                 
sigma = std(x,1)
printmat([mu;sigma])

println("covariance and correlation matrices:")
printmat(cov(x))
printmat(cor(x))


mean and std of random draws: 
    -1.106     9.934
     0.981     1.322

covariance and correlation matrices:
     0.962     0.375
     0.375     1.747

     1.000     0.290
     0.290     1.000



## Quantiles ("critical values") of Distributions

In [10]:
N05     = quantile(Normal(0,1),0.05)            #from the Distributions package
Chisq05 = quantile(Chisq(5),0.95)

println("\n","5th percentile of N(0,1) and 95th of Chisquare(5)")      #lots of statistics functions
printmat([N05 Chisq05])


5th percentile of N(0,1) and 95th of Chisquare(5)
    -1.645    11.070



# Statistical Plots

In [11]:
xTicksLoc = Dates.value.([Date(1980);Date(1990);Date(2000);Date(2010)])
xTicksLab = ["1980";"1990";"2000";"2010"]               #crude way of getting the tick marks right

plot3a = plot(dN,Rme,color=:blue,legend=false,xticks=(xTicksLoc,xTicksLab))
plot!(ylims=(-25,25))
title!("Time series plot: monthly US equity market excess return")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 Time series plot: monthly US equity market excess return 
 
 
 % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 51.5968,174.87 52.9884,227.137 54.2453,164.058 55.6369,203.31 56.9836,218.667 58.3752,176.936 59.722,198.628 61.1136,164.403 62.5052,208.406 63.8519,259.71 
 65.2435,166.675 66.5902,190.777 67.9818,163.989 69.3734,209.095 70.6752,294.761 72.0668,176.316 73.4135,167.846 74.8051,181.894 76.1518,159.513 77.5434,191.81 
 78.935,188.505 80.2817,196.355 81.6733,137.959 83.02,236.365 84.4116,238.431 85.8032,200.349 87.0602,180.172 88.4518,218.874 89.7985,202.209 91.1901,219.975 
 92.5368,214.329 93.9284,251.239 95.32,256.129 96.6667,170.531 98.0583,179.484 99.405,228.997 100.797,227.206 102.188,245.179 103.445,217.359 104.837,181.618 
 106.183,230.374 107.575,226.724 108.922,225.002 110.313,126.941 111.705,195.598 113.052,126.046 114.443,172.253 115.79,198.283 117.182,179.553 118.573,187.128 
 119.83,184.098 121.222,157.447 122.568,199.316 123.96,182.238 125.307,230.512 126.698,206.478 128.09,197.801 129.437,228.17 130.828,188.092 132.175,215.912 
 133.567,217.841 134.958,235.47 136.26,199.454 137.652,207.511 138.998,245.042 140.39,192.705 141.737,223.487 143.128,131.761 144.52,209.302 145.866,210.61 
 147.258,216.05 148.605,191.741 149.996,149.115 151.388,196.011 152.645,209.095 154.037,210.128 155.383,169.774 156.775,195.667 158.122,208.131 159.513,210.748 
 160.905,235.194 162.251,177.555 163.643,160.202 164.99,178.451 166.381,200.762 167.773,157.378 169.03,170.669 170.421,212.676 171.768,172.046 173.16,197.457 
 174.506,248.347 175.898,161.235 177.29,261.156 178.636,172.873 180.028,195.942 181.375,225.209 182.766,118.057 184.158,173.63 185.415,190.571 186.806,218.392 
 188.153,202.76 189.545,176.867 190.891,176.385 192.283,181.343 193.675,221.077 195.021,363.005 196.413,255.853 197.76,157.929 199.151,174.732 200.543,171.22 
 201.845,218.116 203.236,199.247 204.583,206.891 205.975,171.564 207.321,212.194 208.713,226.999 210.105,182.307 211.451,195.735 212.843,218.874 214.19,193.463 
 215.581,161.923 216.973,219.149 218.23,193.463 219.621,175.076 220.968,182.032 222.36,211.918 223.706,155.381 225.098,193.532 226.489,209.164 227.836,228.514 
 229.228,196.149 230.574,195.253 231.966,255.853 233.358,197.319 234.615,191.466 236.006,227.895 237.353,147.118 238.745,210.885 240.091,214.811 241.483,271.485 
 242.874,244.835 244.221,216.945 245.613,162.337 246.959,187.472 248.351,173.424 249.743,154.762 251,186.783 252.391,205.032 253.738,178.864 255.13,236.847 
 256.476,174.801 257.868,188.367 259.259,214.397 260.606,194.289 261.998,232.027 263.344,132.725 264.736,206.822 266.128,196.355 267.429,222.317 268.821,196.631 
 270.168,201.176 271.559,219.149 272.906,178.313 274.298,219.769 275.689,196.906 277.036,197.664 278.428,177.555 279.774,193.325 281.166,196.562 282.557,201.451 
 283.814,188.092 285.206,222.799 286.553,184.786 287.944,201.658 289.291,205.858 290.683,178.175 292.074,205.032 293.421,192.705 294.813,217.496 296.159,191.81 
 297.551,183.684 298.942,221.766 300.199,237.054 301.591,198.972 302.938,199.385 304.329,225.002 305.676,184.511 307.068,176.867 308.459,218.874 309.806,196.286 
 311.198,231.82 312.544,198.008 313.936,192.499 315.327,179.139 316.584,188.229 317.976,189.469 319.323,183.96 320.714,185.406 322.061,178.657 323.453,200.487 
 324.844,181.55 326.191,214.673 327.582,177.142 328.929,196.562 330.321,187.265 331.712,195.116 333.014,198.834 334.406,189.262 335.753,188.092 337.144,212.125 
 338.491,243.802 339.882,184.098 341.274,170.187 342.621,197.113 344.012,161.304 345.359,214.673 346.751,169.912 348.142,207.098 349.399,237.536 350.791,177.418 
 352.137,157.723 353.529,175.834 354.876,153.935 356.267,231.476 357.659,166.4 359.006,230

In [12]:
plot3b = scatter(Rme,Re,color=:blue,legend=false)
plot!([-40;60],[-40;60],color=:black)
plot!(xlims=(-40,40),ylims=(-40,60))
title!("Scatter plot: two monthly return series (and 45 degree line)")
xlabel!("Market excess return, %")
ylabel!("Excess returns on small growth stocks, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -40 
 
 
 -20 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 -40 
 
 
 -20 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 Scatter plot: two monthly return series (and 45 degree line) 
 
 
 Market excess return, % 
 
 
 Excess returns on small growth stocks, %

In [13]:
xGrid = -25:0.1:15
pdfX   = pdf(Normal(mean(Rme),std(Rme)),xGrid) #the N(μ,σ) pdf
                                        #"Distributions" wants σ, not σ^2

histogram(Rme,bins = -25:1:15,normalized=true,label="histogram")     #normalized to have area=1
plot!(xGrid,pdfX,linewidth=3,label="fitted N()")
title!("Histogram: monthly US equity market excess return")
xlabel!("Market excess return, %")
ylabel!("")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 Histogram: monthly US equity market excess return 
 
 
 Market excess return, % 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:3; stroke-opacity:1; fill:none" points="
 54.1373,360.065 55.4144,360.065 56.6916,360.065 57.9687,360.065 59.2458,360.065 60.5229,360.065 61.8001,360.065 63.0772,360.065 64.3543,360.065 65.6315,360.065 
 66.9086,360.065 68.1857,360.065 69.4629,360.065 70.74,360.065 72.0171,360.065 73.2943,360.065 74.5714,360.065 75.8485,360.065 77.1256,360.065 78.4028,360.065 
 79.6799,360.065 80.957,360.065 82.2342,360.065 83.5113,360.065 84.7884,360.065 86.0656,360.064 87.3427,360.064 88.6198,360.064 89.8969,360.064 91.1741,360.064 
 92.4512,360.064 93.7283,360.064 95.0055,360.063 96.2826,360.063 97.5597,360.063 98.8369,360.063 100.114,360.062 101.391,360.062 102.668,360.062 103.945,360.061 
 105.223,360.061 106.5,360.06 107.777,360.06 109.054,360.059 110.331,360.059 111.608,360.058 112.885,360.057 114.162,360.056 115.44,360.055 116.717,360.054 
 117.994,360.053 119.271,360.052 120.548,360.051 121.825,360.049 123.102,360.048 124.379,360.046 125.657,360.044 126.934,360.042 128.211,360.04 129.488,360.037 
 130.765,360.034 132.042,360.031 133.319,360.028 134.597,360.024 135.874,360.02 137.151,360.016 138.428,360.012 139.705,360.006 140.982,360.001 142.259,359.995 
 143.536,359.989 144.814,359.982 146.091,359.974 147.368,359.966 148.645,359.957 149.922,359.947 151.199,359.936 152.476,359.925 153.753,359.913 155.031,359.899 
 156.308,359.885 157.585,359.869 158.862,359.852 160.139,359.834 161.416,359.815 162.693,359.794 163.97,359.771 165.248,359.746 166.525,359.72 167.802,359.692 
 169.079,359.661 170.356,359.628 171.633,359.593 172.91,359.555 174.188,359.514 175.465,359.471 176.742,359.424 178.019,359.374 179.296,359.32 180.573,359.262 
 181.85,359.201 183.127,359.135 184.405,359.065 185.682,358.99 186.959,358.909 188.236,358.824 189.513,358.732 190.79,358.635 192.067,358.531 193.344,358.42 
 194.622,358.303 195.899,358.177 197.176,358.044 198.453,357.902 199.73,357.752 201.007,357.592 202.284,357.423 203.562,357.243 204.839,357.052 206.116,356.851 
 207.393,356.637 208.67,356.41 209.947,356.171 211.224,355.918 212.501,355.65 213.779,355.367 215.056,355.069 216.333,354.754 217.61,354.422 218.887,354.072 
 220.164,353.704 221.441,353.316 222.718,352.907 223.996,352.478 225.273,352.026 226.55,351.552 227.827,351.054 229.104,350.531 230.381,349.983 231.658,349.408 
 232.936,348.806 234.213,348.175 235.49,347.515 236.767,346.824 238.044,346.103 239.321,345.348 240.598,344.561 241.875,343.739 243.153,342.881 244.43,341.987 
 245.707,341.055 246.984,340.085 248.261,339.075 249.538,338.024 250.815,336.932 252.092,335.797 253.37,334.619 254.647,333.395 255.924,332.127 257.201,330.811 
 258.478,329.448 259.755,328.037 261.032,326.577 262.31,325.066 263.587,323.504 264.864,321.891 266.141,320.226 267.418,318.507 268.695,316.735 269.972,314.909 
 271.249,313.027 272.527,311.091 273.804,309.099 275.081,307.05 276.358,304.946 277.635,302.785 278.912,300.567 280.189,298.293 281.466,295.961 282.744,293.574 
 284.021,291.13 285.298,288.629 286.575,286.073 287.852,283.462 289.129,280.796 290.406,278.076 291.684,275.302 292.961,272.476 294.238,269.598 295.515,266.67 
 296.792,263.693 298.069,260.668 299.346,257.596 300.623,254.479 301.901,251.318 303.178,248.116 304.455,244.873 305.732,241.593 307.009,238.277 308.286,234.927 
 309.563,231.545 310.84,228.134 312.118,224.697 313.395,221.236 314.672,217.753 315.949,214.252 317.226,210.735 318.503,207.205 319.78,203.666 321.058,200.12 
 322.335,196.571 323.612,193.022 324.889,189.477 326

In [14]:
println("\n","end of program")


end of program
